# Main functions
 - StandardFileReader
 - maybeFST
 - get_basic_dataframe
 - StandardFileWriter
 - add_dask_column
 - add_grid_column
 - add_columns
 - compute
 - get_2d_lat_lon_df
 - select_with_meta
 - metadata_cleanup
 - to_dask
 - to_numpy
 - unit_convert
 - unit_convert_array
 - fststat
 - voir

In [1]:
import pandas as pd
import os
import multiprocessing as mp
import numpy.ma as ma
import datetime
import fstpy
import glob
from os import getenv

pd.options.mode.chained_assignment = None  # default='warn'

CMCGRIDF = "/space/hall5/sitestore/eccc/prod/hubs/gridpt/dbase"
ATM_MODEL_DFILES = "/fs/ssm/eccc/mrd/rpn/MIG/GEM/d/gem-data/gem-data_4.2.0/gem-data_4.2.0_all/share/data/dfiles"

# StandardFileReader and maybeFST
- reads in the record information of the provided file(s)

In [2]:
print(f"StandardFileReader:{fstpy.StandardFileReader.__doc__}")
print(f"maybeFST:{fstpy.maybeFST.__doc__}")
directory = os.path.join(ATM_MODEL_DFILES, "bcmk", "**")
files = glob.glob(directory)
# check that the files are FST files
files = [f for f in files[:10] if fstpy.maybeFST(f)]
df = fstpy.StandardFileReader(files).to_pandas()
cols = list(df.columns)
cols.remove("d")
df[cols]

StandardFileReader:Class to handle fst files. Opens, reads the contents of an fst file or files into a pandas dataframe and closes. Extra metadata columns are added to the dataframe if specified.    

        :param filenames: path to file or list of paths to files  
        :type filenames: str|pathlib.Path|list[str], does not accept wildcards (numpy has 
                         many tools for this)  
        :param decode_metadata: adds extra columns, defaults to False  
            'unit':str, unit name   
            'unit_converted':bool  
            'description':str, field description   
            'date_of_observation':datetime, of the date of observation   
            'date_of_validity': datetime, of the date of validity   
            'level':float32, decoded ip1 level   
            'ip1_kind':int32, decoded ip1 kind   
            'ip1_pkind':str, string repr of ip1_kind int   
            'data_type_str':str, string repr of data type   
            'label':str, label d

,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,npas,datyp,nbits,grtyp,ig1,ig2,ig3,ig4,datev,grid
0,P0,P,G133K80P,200,100,1,354514400,0,0,0,...,0,1,12,G,0,0,0,0,354514400,00
1,TT,P,G133K80P,200,100,1,354514400,97642568,0,0,...,0,1,12,G,0,0,0,0,354514400,00
2,TT,P,G133K80P,200,100,1,354514400,97738568,0,0,...,0,1,12,G,0,0,0,0,354514400,00
3,TT,P,G133K80P,200,100,1,354514400,97899568,0,0,...,0,1,12,G,0,0,0,0,354514400,00
4,TT,P,G133K80P,200,100,1,354514400,98152568,0,0,...,0,1,12,G,0,0,0,0,354514400,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,I8,P,CCCMACISLDEO,200,100,1,323427143,0,8,0,...,0,1,12,G,0,0,0,0,323427143,00
1583,I8,P,CCCMACISLDEO,200,100,1,323427143,0,9,0,...,0,1,12,G,0,0,0,0,323427143,00
1584,I8,P,CCCMACISLDEO,200,100,1,323427143,0,10,0,...,0,1,12,G,0,0,0,0,323427143,00
1585,I8,P,CCCMACISLDEO,200,100,1,323427143,0,11,0,...,0,1,12,G,0,0,0,0,323427143,00


# Get a lot of files

In [3]:
fdate = datetime.date.today().strftime("%Y%m%d") + "**"

directory = os.path.join(CMCGRIDF, "prog", "glbpres.usr", fdate)

files = glob.glob(directory)

with mp.Pool(processes=20) as pool:
    res = pool.map(fstpy.maybeFST, [file for file in files])  # runs in *only* one process

files = list(ma.masked_array(files, mask=[not elem for elem in res]))

print(f"found {len(files)} FST files")

found 137 FST files


# get_basic_dataframe (lightweight reader)

In [4]:
print(f"get_basic_dataframe:{fstpy.get_basic_dataframe.__doc__}")

with mp.Pool(processes=int(mp.cpu_count() / 2)) as pool:
    df_list = pool.map(fstpy.get_basic_dataframe, [file for file in files])  # runs in *only* one process

df = pd.concat(df_list, ignore_index=True)
df

get_basic_dataframe:Creates a dataframe of all non deleted records in an FST file, does not include data 'd'

    :param path: path of file to load
    :type path: str
    :return: dataframe of all non deleted records in an FST file
    :rtype: pd.DataFrame
    


,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,ig1,ig2,ig3,ig4,datev,lng,swa,key,path,shape
0,>>,X,G1_9_0_0N,1801,1,1,0,68839,90098,0,...,900,0,43200,43200,0,1826,2335,1,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1)"
1,^^,X,G1_9_0_0N,1,1251,1,0,68839,90098,0,...,900,0,43200,43200,0,1276,3248,1025,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1, 1251)"
2,ES,P,G1_9_0_0N,1801,1251,1,477958400,900,3,0,...,68839,90098,0,0,477961100,849436,3886,2049,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
3,ES,P,G1_9_0_0N,1801,1251,1,477958400,985,3,0,...,68839,90098,0,0,477961100,830978,428604,3073,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
4,ES,P,G1_9_0_0N,1801,1251,1,477958400,20,3,0,...,68839,90098,0,0,477961100,145388,844093,4097,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29252,TT,P,G1_9_0_0N,1801,1251,1,477958400,93423264,52,0,...,68839,90098,0,0,478005200,502368,1850687,9217,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
29253,TD,P,G1_9_0_0N,1801,1251,1,477958400,93423264,52,0,...,68839,90098,0,0,478005200,284962,2101871,10241,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
29254,PR,P,G1_9_0_0N,1801,1251,1,477958400,0,52,0,...,68839,90098,0,0,478005200,1762182,2244352,11265,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"
29255,PC,P,G1_9_0_0N,1801,1251,1,477958400,0,52,0,...,68839,90098,0,0,478005200,958074,3125443,12289,/space/hall5/sitestore/eccc/prod/hubs/gridpt/d...,"(1801, 1251)"


# StandardFileWriter, add_grid_column, add_dask_column

In [5]:
# setup our output file
user = getenv("USER")
out_file = f"/home/{user}/TT.std"
if os.path.isfile(out_file):
    os.remove(out_file)


print(f"StandardFileWriter:{fstpy.StandardFileWriter.__doc__}")
print(f"add_grid_column:{fstpy.add_grid_column.__doc__}")
print(f"add_dask_column:{fstpy.add_dask_column.__doc__}")

# the basic dataframe has no grid column
df = fstpy.add_grid_column(df)

# get all the TT
tt_df = df.loc[(df.nomvar == "TT")]

# get the first grid
grid = tt_df.iloc[0].grid

# select TT with first grid only
tt_df = tt_df.loc[(tt_df.grid == grid)]

# get TT's horizontal and vertical grid fields
grid_meta_df = df.loc[(df.nomvar.isin([">>", "^^", "^>", "!!", "PT"])) & (df.grid == grid)]
p0_meta_df = df.loc[(df.nomvar.isin(["P0"])) & (df.grid == grid)]


# join all the rows together
tt_df = pd.concat([tt_df, grid_meta_df, p0_meta_df], ignore_index=True)

# cleanup the dataframe
tt_df = tt_df.drop_duplicates(
    subset=[
        "nomvar",
        "typvar",
        "etiket",
        "ni",
        "nj",
        "nk",
        "dateo",
        "ip1",
        "ip2",
        "ip3",
        "deet",
        "npas",
        "datyp",
        "nbits",
        "grtyp",
        "ig1",
        "ig2",
        "ig3",
        "ig4",
        "datev",
    ]
)

# the basic dataframe needs the 'd' column
tt_df = fstpy.add_dask_column(tt_df)  # attention on dois changer le dateo pour la date courante

# write the results
fstpy.StandardFileWriter(out_file, tt_df).to_fst()

# check the results
display(fstpy.get_basic_dataframe(out_file))

os.remove(out_file)

StandardFileWriter:Writes a standard file Dataframe to file. If no metada fields like ^^ and >> are found,
    an attempt will be made to load them from the original file so that they can be added to the output if not already present

    :param filename: path of file to write to
    :type filename: str
    :param df: dataframe to write
    :type df: pd.DataFrame
    :param mode: In 'dump' mode, no processing will be done on the dataframe 
                before writing, data must be present in the dataframe (df = compute(df)).
                If set to 'update', path must be an existing file. Only the 
                field metadata will be updated, the data itself will not be 
                modified. In 'write' mode, the data will be loaded, metadata 
                fields like '>>' will be added if not present default 'write'
    :type mode: str
    :param no_meta: if true these fields ["^>", ">>", "^^", "!!", "!!SF", "HY", "P0", "PT", "E1"] will be removed from the dataframe
   

,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,ig1,ig2,ig3,ig4,datev,lng,swa,key,path,shape
0,^^,X,G1_9_0_0N,1,1251,1,0,68839,90098,0,...,900,0,43200,43200,0,1276,2335,6,/home/sbf000/TT.std,"(1, 1251)"
1,>>,X,G1_9_0_0N,1801,1,1,0,68839,90098,0,...,900,0,43200,43200,0,1826,2973,1030,/home/sbf000/TT.std,"(1801, 1)"
2,P0,P,G1_9_0_0N,1801,1251,1,477958400,0,3,0,...,68839,90098,0,0,477961100,932470,3886,2054,/home/sbf000/TT.std,"(1801, 1251)"
3,P0,P,G1_9_0_0N,1801,1251,1,477958400,0,4,0,...,68839,90098,0,0,477962000,932646,470121,3078,/home/sbf000/TT.std,"(1801, 1251)"
4,P0,P,G1_9_0_0N,1801,1251,1,477958400,0,2,0,...,68839,90098,0,0,477960200,932208,936444,4102,/home/sbf000/TT.std,"(1801, 1251)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3998,TT,P,G1_9_0_0N,1801,1251,1,477958400,1660,240,0,...,68839,90098,0,0,478174400,301100,862158841,8026118,/home/sbf000/TT.std,"(1801, 1251)"
3999,TT,P,G1_9_0_0N,1801,1251,1,477958400,30,240,0,...,68839,90098,0,0,478174400,366996,862309391,8027142,/home/sbf000/TT.std,"(1801, 1251)"
4000,TT,P,G1_9_0_0N,1801,1251,1,477958400,93423264,234,0,...,68839,90098,0,0,478169000,501696,862492889,8028166,/home/sbf000/TT.std,"(1801, 1251)"
4001,TT,P,G1_9_0_0N,1801,1251,1,477958400,93423264,237,0,...,68839,90098,0,0,478171700,503064,862743737,8029190,/home/sbf000/TT.std,"(1801, 1251)"


# add_columns

In [6]:
print(f"add_columns:{fstpy.add_columns.__doc__}")

df = fstpy.StandardFileReader(
    "/home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std"
).to_pandas()

print(df.columns)

orig_cols = set(list(df.columns))

# add decode ips
df = fstpy.add_columns(df, "ip_info")

new_cols = set(list(df.columns))

print(df.columns)

diff = new_cols.difference(orig_cols)

print(f"newly added columns {diff}")

df[list(diff)]

add_columns:If valid columns are provided, they will be added. 
       These include ['flags','etiket','unit','dateo','datev','forecast_hour', 'datyp','ip_info', 'description']
       Replaces original column(s) if present.   

    :param df: dataframe to modify (meta data needs to be present in dataframe)
    :type df: pd.DataFrame
    :param decode: if decode is True, add the specified columns
    :type decode: bool
    :param columns: [description], defaults to  ['flags','etiket','unit','dateo','datev','forecast_hour', 'datyp','ip_info', 'description']
    :type columns: list[str], optional
    
Index(['nomvar', 'typvar', 'etiket', 'ni', 'nj', 'nk', 'dateo', 'ip1', 'ip2',
       'ip3', 'deet', 'npas', 'datyp', 'nbits', 'grtyp', 'ig1', 'ig2', 'ig3',
       'ig4', 'datev', 'grid', 'd'],
      dtype='object')
Index(['nomvar', 'typvar', 'etiket', 'ni', 'nj', 'nk', 'dateo', 'ip1', 'ip2',
       'ip3', 'deet', 'npas', 'datyp', 'nbits', 'grtyp', 'ig1', 'ig2', 'ig3',
       'ig4', 'datev', 

,ip1_kind,ip3_kind,ip3_pkind,ip2_kind,follow_topography,surface,ip3_dec,interval,ip2_pkind,vctype,ip2_dec,ascending,level,ip1_pkind
0,1,100,,10,True,True,0.0,None,H,UNKNOWN,0.0,False,1.0,sg
1,1,100,,10,True,True,0.0,None,H,UNKNOWN,0.0,False,1.0,sg
2,1,100,,10,True,True,0.0,None,H,UNKNOWN,0.0,False,1.0,sg
3,100,100,,100,False,False,0.0,None,,UNKNOWN,235.0,True,153.0,
4,100,100,,100,False,False,0.0,None,,UNKNOWN,235.0,True,153.0,


# compute

In [7]:
print(f"compute:{fstpy.compute.__doc__}")

df = fstpy.StandardFileReader(
    "/home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std"
).to_pandas()

df = df.loc[df.nomvar == "TT"]

tt_df = pd.DataFrame([df.iloc[0].to_dict()])

print(f"dask array {tt_df.iloc[0].d}")

tt_df = fstpy.compute(tt_df)

print(f"numpy array {tt_df.iloc[0].d}")

compute:Converts all dask arrays contained in the 'd' column, by numpy arrays

    :param df: input DataFrame
    :type df: pd.DataFrame
    :param remove_path_and_key: remove path and key column after conversion, defaults to True
    :type remove_path_and_key: bool, optional
    :return: modified dataframe with numpy arrays instead of dask arrays
    :rtype: pd.DataFrame
    
dask array dask.array</home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std:2054, shape=(576, 641), dtype=float32, chunksize=(576, 641), chunktype=numpy.ndarray>
numpy array [[27.4581   27.23349  26.518646 ... 28.16513  27.629974 24.954193]
 [27.42685  27.182709 26.307709 ... 28.219818 27.465912 25.012787]
 [27.409271 27.026459 26.288177 ... 28.38388  27.35263  25.17099 ]
 ...
 [27.450287 27.190521 26.47763  ... 27.260834 28.047943 24.735443]
 [27.462006 27.200287 26.479584 ... 28.182709 27.817474 24.79013 ]
 [27.4581   27.23349  26.518646 ... 28.16513  27.629

# get_2d_lat_lon_df

In [8]:
print(f"get_2d_lat_lon_df:{fstpy.get_2d_lat_lon_df.__doc__}")

df = fstpy.StandardFileReader(
    "/home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std"
).to_pandas()

tt_df = df.loc[df.nomvar == "TT"]

lat_lon_df = fstpy.get_2d_lat_lon_df(tt_df)

lat_lon_df

get_2d_lat_lon_df:Gets the latitudes and longitudes as 2d arrays associated with the supplied grids

    :return: a pandas Dataframe object containing the lat and lon meta data of the grids
    :rtype: pd.DataFrame
    :raises Get2DLatLonError: no records to process
    


,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,grtyp,ig1,ig2,ig3,ig4,datev,grid,d,path,key
0,LA,P,R1558V0N,576,641,1,353315600,12000,0,0,...,Z,153,235,0,0,353315600,153235,"[[-7.8338194, -10.389295, -12.936791, -15.4741...",/home/spst900/data/sitestore6/spooki/spooki_di...,2055
1,LO,P,R1558V0N,576,641,1,353315600,12000,0,0,...,Z,153,235,0,0,353315600,153235,"[[340.01065, 341.60236, 343.22028, 344.87155, ...",/home/spst900/data/sitestore6/spooki/spooki_di...,2055


# select_with_meta

In [9]:
print(f"select_with_meta:{fstpy.select_with_meta.__doc__}")

df = fstpy.StandardFileReader(
    "/home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std"
).to_pandas()

uuvv_df = fstpy.select_with_meta(df, ["UU", "VV"])

uuvv_df

select_with_meta:Select fields with accompaning meta data  

    :param df: dataframe to select from  
    :type df: pd.DataFrame  
    :param nomvar: list of nomvars to select   
    :type nomvar: list  
    :raises SelectError: if dataframe is empty, if nothing to select or if variable not found in dataframe  
    :return: dataframe with selection results  
    :rtype: pd.DataFrame  
    


,nomvar,typvar,etiket,ni,nj,nk,dateo,ip1,ip2,ip3,...,ip2_kind,ip2_pkind,ip3_dec,ip3_kind,ip3_pkind,surface,follow_topography,ascending,interval,vctype
0,^^,X,R1558V0N,1,641,1,353315600,153,235,0,...,100,,0.0,100,,False,False,True,None,UNKNOWN
1,>>,X,R1558V0N,576,1,1,353315600,153,235,0,...,100,,0.0,100,,False,False,True,None,UNKNOWN
2,UU,P,R1558V0N,576,641,1,353315600,12000,0,0,...,10,H,0.0,100,,True,True,False,None,UNKNOWN
3,VV,P,R1558V0N,576,641,1,353315600,12000,0,0,...,10,H,0.0,100,,True,True,False,None,UNKNOWN


# metadata_cleanup

In [10]:
print(f"metadata_cleanup:{fstpy.metadata_cleanup.__doc__}")

df = fstpy.StandardFileReader(
    "/home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std"
).to_pandas()

meta_df = df.loc[df.nomvar.isin(["!!", "^^", ">>", "^>", "P0", "PT"])]

uuvv_df = fstpy.select_with_meta(df, ["UU", "VV"])

all_df = pd.concat([meta_df, uuvv_df], ignore_index=True)

print(
    f"before cleanup{all_df.loc[all_df.nomvar.isin(['!!', '^^', '>>', '^>', 'P0', 'PT'])][['nomvar', 'typvar', 'etiket', 'ni', 'nj', 'nk', 'dateo', 'ip1', 'ip2', 'ip3', 'deet', 'npas', 'datyp', 'nbits', 'grtyp', 'ig1', 'ig2', 'ig3', 'ig4', 'datev']].to_string()}"
)

all_df = fstpy.metadata_cleanup(all_df)

print(
    f"after cleanup{all_df.loc[all_df.nomvar.isin(['!!', '^^', '>>', '^>', 'P0', 'PT'])][['nomvar', 'typvar', 'etiket', 'ni', 'nj', 'nk', 'dateo', 'ip1', 'ip2', 'ip3', 'deet', 'npas', 'datyp', 'nbits', 'grtyp', 'ig1', 'ig2', 'ig3', 'ig4', 'datev']].to_string()}"
)

metadata_cleanup:Cleans the metadata from a dataframe according to rules.   

    :param df: dataframe to clean  
    :type df: pd.DataFrame  
    :return: dataframe with only cleaned meta_data  
    :rtype: pd.DataFrame  
    
before cleanup  nomvar typvar    etiket   ni   nj  nk      dateo  ip1  ip2  ip3  deet  npas  datyp  nbits grtyp   ig1  ig2    ig3    ig4      datev
0     >>      X  R1558V0N  576    1   1  353315600  153  235    0   450     0      5     32     E  1480  750  56000  44000  353315600
1     ^^      X  R1558V0N    1  641   1  353315600  153  235    0   450     0      5     32     E  1480  750  56000  44000  353315600
2     ^^      X  R1558V0N    1  641   1  353315600  153  235    0   450     0      5     32     E  1480  750  56000  44000  353315600
3     >>      X  R1558V0N  576    1   1  353315600  153  235    0   450     0      5     32     E  1480  750  56000  44000  353315600
after cleanup  nomvar typvar    etiket   ni   nj  nk      dateo  ip1  ip2  ip3  deet  np

# to_dask / to_numpy

In [11]:
print(f"to_dask:{fstpy.to_dask.__doc__}")
print(f"to_numpy:{fstpy.to_numpy.__doc__}")

# get the data of the 0 row in the dataframe
dask_array = all_df.loc[0].d

print(dask_array)

print(fstpy.to_dask(dask_array))

numpy_array = fstpy.to_numpy(dask_array)

print(numpy_array)

print(fstpy.to_dask(numpy_array))

to_dask:If the array is of dask type, no op, else comvert array to dask array

    :param arr: array to convert
    :type arr: np.ndarray|da.core.Array
    :raises ConversionError: Raised if not a numpy or dask array
    :return: a dask array
    :rtype: da.core.Array
    
to_numpy:If the array is of numpy type, no op, else compute de daks array to get a numpy array

    :param arr: array to convert
    :type arr: np.ndarray|da.core.Array
    :raises ConversionError: Raised if not a numpy or dask array
    :return: a numpy array
    :rtype: np.ndarray
    
dask.array</home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std:1031, shape=(1, 641), dtype=float32, chunksize=(1, 641), chunktype=numpy.ndarray>
dask.array</home/spst900/data/sitestore6/spooki/spooki_dir/pluginsRelatedStuff/WindChill/testsFiles/UUVVTT_fileSrc.std:1031, shape=(1, 641), dtype=float32, chunksize=(1, 641), chunktype=numpy.ndarray>
[[-88.5947     -85.5947     -82.594

# Unit Conversion with cf_units

In [12]:
print(f"Unit conversion in fstpy uses the cf_units library for robust and standardized unit conversions.")
print("The system provides:")
print("1. Direct conversion between compatible units using cf_units")
print("2. Automatic mapping of CMC unit names to cf_units compatible names")
print("3. Support for common meteorological unit conversions")

# Example of unit conversion using cf_units
from fstpy.unit_helpers import get_cf_unit

# Get a cf_units Unit object for knots
knots = get_cf_unit("knot")
print(f"Knots unit: {knots}")

# Get a cf_units Unit object for meters per second
mps = get_cf_unit("meter_per_second")
print(f"Meters per second unit: {mps}")

# Convert a value from knots to meters per second
value_in_knots = 50.0
value_in_mps = knots.convert(value_in_knots, mps)
print(f"{value_in_knots} knots = {value_in_mps} m/s")

# Example with temperature
celsius = get_cf_unit("celsius")
kelvin = get_cf_unit("kelvin")
temp_c = 25.0
temp_k = celsius.convert(temp_c, kelvin)
print(f"{temp_c}°C = {temp_k}K")

Unit conversion in fstpy uses the cf_units library for robust and standardized unit conversions.
The system provides:
1. Direct conversion between compatible units using cf_units
2. Automatic mapping of CMC unit names to cf_units compatible names
3. Support for common meteorological unit conversions
Knots unit: knot
Meters per second unit: m/s
50.0 knots = 25.722222222222225 m/s
25.0°C = 298.15K


# fststat

In [13]:
print(f"fststat:{fstpy.fststat.__doc__}")

fstpy.fststat(all_df.loc[all_df.nomvar == "UU"])

fststat:Produces summary statistics for a dataframe

    :param df: input dataframe
    :type df: pd.DataFrame
    
  nomvar typvar        level    ip1  ip2  ip3      dateo    etiket          mean          std     min_pos           min     max_pos          max
2     UU      P 1.000000E+00  12000    0    0  353315600  R1558V0N -1.045754E-01 1.271628E+01  (462, 197) -4.724017E+01  (460, 156) 5.834772E+01


# voir

In [14]:
print(f"voir{fstpy.voir.__doc__}")

fstpy.voir(all_df)

voirDisplays the metadata of the supplied records in the rpn voir format

  nomvar typvar    etiket   ni   nj  nk               dateo    ip1  ip2  ip3  deet  npas datyp  nbits grtyp   ig1  ig2    ig3    ig4
0     >>      X  R1558V0N  576    1   1 2009-03-02 12:00:00    153  235    0   450     0     E     32     E  1480  750  56000  44000
1     UU      P  R1558V0N  576  641   1 2009-03-02 12:00:00  12000    0    0   450     0     R     16     Z   153  235      0      0
2     VV      P  R1558V0N  576  641   1 2009-03-02 12:00:00  12000    0    0   450     0     R     16     Z   153  235      0      0
3     ^^      X  R1558V0N    1  641   1 2009-03-02 12:00:00    153  235    0   450     0     E     32     E  1480  750  56000  44000
